In [1]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/mmfs1/gscratch/ark/chan0369/ark/py3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
tqdm.pandas()

In [8]:
df = pd.read_csv('../meaning.csv')
df = df[:100]
df['kor_equ'] = ''
df.head()

,idiom,meaning,kor_equ
0,get to [pron] point,get to the main point of a discussion or conve...,
1,bring (somebody) to ([pron]) knees,force someone to submit to one's will or power.,
2,make up [pron] mind,come to a decision or opinion.,
3,build bridges,create a connection or relationship with someone.,
4,raise eyebrows,express surprise or disapproval.,


In [9]:
dfs_lang = [
    pd.read_csv('./kor/idioms_korean.csv'),
    pd.read_csv('./chn/idioms_chinese.csv'),
]
langs = ['kor', 'chn']
dfs_lang[0].head()

,idiom,meaning
0,가락이 나다,hit one's stride
1,가랑이가 찢어지게 가난하다,to suffer from extreme poverty
2,가로 지나 세로 지나,this way or that way
3,가면을 벗다,to show one's true self
4,가면을 쓰다,to lie or pretend


In [10]:
#Compute embedding for both lists

for i in range(len(langs)):
    df_lang = dfs_lang[i]
    def get_similarity(sentence1, sentence2):
        embedding_1= model.encode(sentence1, convert_to_tensor=True)
        embedding_2 = model.encode(sentence2, convert_to_tensor=True)
        p = float(util.pytorch_cos_sim(embedding_1, embedding_2)[0][0])
        return p
    def get_equivalence(query):
        p_arr = df_lang['meaning'].apply(lambda key: get_similarity(query, key))
        max_p = np.max(p_arr)
        if max_p < 0.7: return ''
        value = df_lang['idiom'][np.argmax(p_arr)]
        return value
    df[langs[i]] = df['meaning'].progress_apply(get_equivalence)

df.head()

100%|██████████| 100/100 [03:42<00:00,  2.23s/it]


,idiom,meaning,kor_equ,kor,chn
0,get to [pron] point,get to the main point of a discussion or conve...,,,
1,bring (somebody) to ([pron]) knees,force someone to submit to one's will or power.,,,
2,make up [pron] mind,come to a decision or opinion.,,,
3,build bridges,create a connection or relationship with someone.,,,
4,raise eyebrows,express surprise or disapproval.,,,


In [13]:
df

,idiom,meaning,kor_equ,kor,chn
0,get to [pron] point,get to the main point of a discussion or conve...,,,
1,bring (somebody) to ([pron]) knees,force someone to submit to one's will or power.,,,
2,make up [pron] mind,come to a decision or opinion.,,,
3,build bridges,create a connection or relationship with someone.,,,
4,raise eyebrows,express surprise or disapproval.,,,
...,...,...,...,...,...
95,sleep on [pron],think about something for a while before makin...,,,
96,put on [pron] brave face,try to appear strong and confident even when f...,,,
97,cross [pron] fingers,hope for the best.,,,
98,nurse [pron] back to health,help someone recover from an illness or injury.,,,
